In [41]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### Feature Store

In [42]:
selected_features = [
    'music name', 
    'music id',
    'acousticness',
    'danceability',
    'energy',
    'instrumentalness',
    'liveness',
    'loudness',
    'speechiness',
    'tempo',
    'valence',
    'duration_ms',
    'key',
    'mode',
    'time_signature'
]

### Authentication to Spotify API

In [43]:
with open('credentials.json', 'r') as file:
    credentials = json.load(file)
# 特徴量を格納するための空のリスト
features_list = []

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=credentials['client_id'],
                                               client_secret=credentials['client_secret'],
                                               redirect_uri=credentials['redirect_uri'],
                                               scope='user-read-recently-played'))

### Set Playlist ID

In [44]:
## ②プレイリスト
playlist_id = credentials['train_playlist_id']

### Storing Feature Param

In [45]:
offset = 0
while True:
    tracks = sp.playlist_tracks(playlist_id, offset=offset, fields='items.track.id,items.track.name,total')
    if len(tracks['items']) == 0:
        break

    for track in tracks['items']:
        track_id = track['track']['id']
        track_name = track['track']['name']

        # 各トラックIDに対して特徴量を取得
        features = sp.audio_features(track_id)[0]
        features_list.append([track_name,track_id,features])
        # features_list.append(features) 

    offset += len(tracks['items'])

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0562v4WyycueQ6BIL8X5Ps:
 Max Retries, reason: too many 429 error responses

#### music name addition

In [39]:
df = pd.DataFrame(features_list, columns=['music name', 'music id', 'features'])

# 特徴量の辞書を展開して新たなカラムを作成
features_df = df['features'].apply(pd.Series)

# 元のDataFrameと特徴量のDataFrameを結合
df = pd.concat([df[['music name', 'music id']], features_df], axis=1)
selected_features_df=df[selected_features]
selected_features_df

,music name,music id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,key,mode,time_signature
0,scorching heart,4cHcx4XiWbFX4sh0ou95Is,0.010800,0.587,0.962,0.011900,0.1400,-3.396,0.0857,135.945,0.593,279693,7,0,4
1,革命前夜 -The Eve of the Revolution-,7fsasfFV5KxqDciMceoaIH,0.031500,0.517,0.779,0.000000,0.3610,-5.238,0.0459,140.982,0.433,235307,6,0,4
2,キセキ,2NmPjaxOlVMYl4kPId1ZNC,0.000585,0.522,0.990,0.003290,0.3500,-2.933,0.0422,130.006,0.740,275693,0,1,4
3,"”友達 以上×敵 未満""",1He12CVIMk1S2QSO30tWDV,0.007460,0.556,0.986,0.000000,0.2820,-3.994,0.1080,135.051,0.513,272827,1,1,4
4,VioletPhantom//Innocentelegy,2PEstJDrvAFoJ5aIO6iwNf,0.062100,0.547,0.910,0.000001,0.0792,-3.068,0.0465,130.061,0.451,229385,9,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,Endless Dream,7lj4lCCfOn7KBk2vL2Ncyw,0.001190,0.548,0.898,0.336000,0.0740,-4.730,0.0366,132.039,0.509,276453,7,1,4
1475,New World,0cZWivLZMr2SoozpCuAUua,0.006280,0.591,0.979,0.000230,0.3450,-3.560,0.1590,135.031,0.512,230707,0,1,4
1476,CASTAWAY,6j97K2AfpeRumQ5iXH0mKN,0.111000,0.604,0.722,0.000000,0.1630,-4.485,0.0376,127.999,0.335,246293,5,1,4
1477,visualize,3plHoYr8YlUDSq2aroVmVS,0.000629,0.509,0.950,0.000043,0.0700,-4.249,0.1100,145.948,0.336,241040,11,0,4


In [40]:
# CSV保存
selected_features_df.to_csv("./train/train.csv", index=False)

In [38]:
selected_features_df.head()

,music name,music id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,key,mode,time_signature
0,scorching heart,4cHcx4XiWbFX4sh0ou95Is,0.010800,0.587,0.962,0.011900,0.1400,-3.396,0.0857,135.945,0.593,279693,7,0,4
1,革命前夜 -The Eve of the Revolution-,7fsasfFV5KxqDciMceoaIH,0.031500,0.517,0.779,0.000000,0.3610,-5.238,0.0459,140.982,0.433,235307,6,0,4
2,キセキ,2NmPjaxOlVMYl4kPId1ZNC,0.000585,0.522,0.990,0.003290,0.3500,-2.933,0.0422,130.006,0.740,275693,0,1,4
3,"”友達 以上×敵 未満""",1He12CVIMk1S2QSO30tWDV,0.007460,0.556,0.986,0.000000,0.2820,-3.994,0.1080,135.051,0.513,272827,1,1,4
4,VioletPhantom//Innocentelegy,2PEstJDrvAFoJ5aIO6iwNf,0.062100,0.547,0.910,0.000001,0.0792,-3.068,0.0465,130.061,0.451,229385,9,0,4
